In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO

# Load the YOLO11 model
with torch.no_grad():
    model = YOLO("weights/yolo11x-pose.pt").to('cuda')
model.eval()

# Open the video file
video_path ="videos_test/test.mp4"
cap = cv2.VideoCapture(video_path)


# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:

        
        results = model.track(frame, persist=True,tracker='bytetrack.yaml')

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        keypoints=results[0].keypoints.xy.int().cpu().tolist()

        # Visualize the results on the frame
        #frame= results[0].plot()
        print(track_ids)

        
            

        for (b,id,ky) in zip(boxes,track_ids,keypoints):
            cv2.putText(
                        frame,str(id),(int(b[0]), int(b[1])),fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,color=(255, 0, 0),thickness=2                   
                   )
    
        int
        for ky in keypoints:
            for k in ky:
                cv2.circle(frame,radius=2,center=(int(k[0]),int(k[1])),color=(255,255,33),thickness=5)


        

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [103]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
box=results[0].boxes.xywh
pose=results[0].keypoints.xy
conf=results[0].keypoints.conf
box=box.to('cuda')
pose=pose.to('cuda')

#removing location info
xy=box[:,:-2]
xy = xy.unsqueeze(1).repeat(1, 17, 1)
mask = (pose == 0).all(dim=-1, keepdim=True)
rslt=pose-xy
rslt=rslt * ~mask
rslt[rslt==-0.000]=0.0

#normalising
wh=box[:,-2:]
wh = wh.unsqueeze(1).repeat(1, 17, 1)
rslt=rslt/wh

#adding confidence
conf = conf.unsqueeze(-1)  
rslt = torch.cat((rslt, conf), dim=-1)
rslt


In [ ]:
import torch.nn.functional as F
normalized = F.normalize(rslt, p=2, dim=-1)
normalized


In [ ]:
for i in range(rslt.shape[0]):
    outer_item = rslt[i]
    print("yesp",outer_item)

In [ ]:
print(rslt.shape)
conf.shape


In [ ]:
rslt.reshape(-1)

In [209]:
data=torch.load("data/non_cheating_Dataset.pth")

In [210]:
data.shape

torch.Size([681, 20, 17, 3])

In [212]:
y=torch.load("Data/non_cheating_ylabel.pth")

In [213]:
y.shape

torch.Size([681])

In [190]:
d=data.reshape(data.shape[0],20,-1)